In [3]:
from agentic_custom.prompt_factory import PromptFactory

In [4]:
# Trivial prompt

class AssistantSystemPrompt(PromptFactory):
    _ROOT_PROMPT = "You are a helpful assistant. Your objective is to record important information about the user's requests and responses."

asp = AssistantSystemPrompt()()
print(asp)

You are a helpful assistant. Your objective is to record important information about the user's requests and responses.


In [5]:
# Plain factory with conditional components

class MemoryAgentSystem(PromptFactory):
    _ROOT_PROMPT = "{memory_role}\n{write_memory_instruction}\n{memory_guidelines}\n\nYou cannot record information about {no_record_target}"
    memory_role = "You are a helpful assistant. Your objective is to record important information about the user's requests and responses."
    memory_max_size = "1000000"

    def memory_guidelines(self, llm_provider: str) -> str:
        return {
            "openai": "<xxxx_openai_xxxx>STUFF</xxxx_openai_xxxx>",
            "anthropic": "#xxxx_anthropic_xxxx\nSTUFF",
        }[llm_provider]

    write_memory_instruction = f"""Write information in the memory using the following format: [timestamp] [user_request] [assistant_response]
You can write at most {memory_max_size} bytes of information in the memory."""


extra_attributes = {
    "no_record_target": "OPENAI_KEYS"
}

msp = MemoryAgentSystem(extra_attributes=extra_attributes)
print(msp(llm_provider="openai"))

You are a helpful assistant. Your objective is to record important information about the user's requests and responses.
Write information in the memory using the following format: [timestamp] [user_request] [assistant_response]
You can write at most 1000000 bytes of information in the memory.
<xxxx_openai_xxxx>STUFF</xxxx_openai_xxxx>

You cannot record information about OPENAI_KEYS


In [6]:
class OperatorSystemPrompt(PromptFactory):
    _ROOT_PROMPT = """You are an operator. Your objective is to record executed stuff.
You have access to memory: {write_memory_instruction}

{operator_execution_loop_instructions}

You cannot execute commads for longer than {exec_timeout}
"""

    operator_execution_loop_instructions = """You run in a loop until the task is completed.
At each iteration, you do the following:
1. You read the memory
2. You decide what to do next
3. You execute the next step"""


extra_attributes = {
    "exec_timeout": "10 seconds"
}
ops = OperatorSystemPrompt(
    prompt_factories=[msp],
    extra_attributes=extra_attributes
)(llm_provider="openai")

print(ops)


You are an operator. Your objective is to record executed stuff.
You have access to memory: Write information in the memory using the following format: [timestamp] [user_request] [assistant_response]
You can write at most 1000000 bytes of information in the memory.

You run in a loop until the task is completed.
At each iteration, you do the following:
1. You read the memory
2. You decide what to do next
3. You execute the next step

You cannot execute commads for longer than 10 seconds

